In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# 1. Read Original Data 

## 1.1 Load Original Dataset

Remember to change the path to read data

In [3]:
df_problem = pd.read_csv('./Data/day_1/round1-day1_problem_data.csv')
burrito_price = float(df_problem['burrito_price'][0])
ingredient_cost = float(df_problem['ingredient_cost'][0])
truck_cost = float(df_problem['truck_cost'][0])
print(f"  - The burritos cost ₲{ingredient_cost} to make and are sold for ₲{burrito_price}. Each truck costs ₲{truck_cost} to use per day.")

df_truck_node = pd.read_csv('./Data/day_1/round1-day1_truck_node_data.csv')
truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in df_truck_node.iterrows()}
truck_spots = truck_coordinates.keys()
print(f"  - There are {len(truck_spots)} available 'truck_spots' or places where a truck can be placed around Burritoville.")

df_demand = pd.read_csv('./Data/day_1/round1-day1_demand_node_data.csv')
buildings, building_names, building_coordinates, demand = gp.multidict({
        row['index']: [row['name'], (float(row['x']), float(row['y'])), float(row['demand'])] for ind,row in df_demand.iterrows()
    })
print(f"  - There are in {len(buildings)} buildings with hungry customers also known as demand nodes.")

df_truck_demand_pair = pd.read_csv('./Data/day_1/round1-day1_demand_truck_data.csv')
building_truck_spot_pairs, distance, scaled_demand = gp.multidict({
        (row['demand_node_index'], row['truck_node_index']): [float(row['distance']), float(row['scaled_demand'])] for ind,row in df_truck_demand_pair.iterrows() if float(row['scaled_demand'])>0# (building, truck_spot): distance, scaled_demand
    })
print(f"  - There are in {len(building_truck_spot_pairs)} pairs of trucks spots and buildings with hungry customers.")




  - The burritos cost ₲5.0 to make and are sold for ₲10.0. Each truck costs ₲250.0 to use per day.
  - There are 15 available 'truck_spots' or places where a truck can be placed around Burritoville.
  - There are in 15 buildings with hungry customers also known as demand nodes.
  - There are in 100 pairs of trucks spots and buildings with hungry customers.


In [4]:
# create list of vehicle_type 
vehicle_types = pd.DataFrame({'vehicle_type': [0, 1, 2]})

# use merge to apply cross-join
df_truck_expanded = df_truck_node.merge(vehicle_types, how='cross')

# rearrange the index of columns
df_truck_expanded=df_truck_expanded[['index', 'vehicle_type', 'x', 'y']]

In [5]:
df_truck_expanded

,index,vehicle_type,x,y
0,truck1,0,112.755064,58.066778
1,truck1,1,112.755064,58.066778
2,truck1,2,112.755064,58.066778
3,truck6,0,56.222222,129.485810
4,truck6,1,56.222222,129.485810
5,truck6,2,56.222222,129.485810
6,truck7,0,113.065684,131.038397
7,truck7,1,113.065684,131.038397
8,truck7,2,113.065684,131.038397
9,truck8,0,164.628607,130.417362


## 1.2 Drawing the map

In [6]:
import plotly.graph_objects as go
from PIL import Image
import requests
from io import BytesIO #try 1

def show_map(buildings, building_names, building_coordinates, demand, truck_coordinates, placed_trucks = []):
    """displays the Burrito Optimization map with labels for open truck locations, buildings with demand, and placed trucks [optional].  This is intended to be used in the Gurobi Days Intro to Modeling course"""

    y_max = 550

    truck_spot_x = [value[0] for key, value in truck_coordinates.items()]
    truck_spot_y = [y_max - value[1] for key, value in truck_coordinates.items()]
    trucks = [key for key, value in truck_coordinates.items()]

    building_x = [value[0] for key, value in building_coordinates.items()]
    building_y = [y_max - value[1] for key, value in building_coordinates.items()]
    demand = [value for key, value in demand.items()]
    building_names = [value for key, value in building_names.items()]

    if placed_trucks:
        placed_truck_spot_x = [value[0] for key, value in truck_coordinates.items() if key in placed_trucks]
        placed_truck_spot_y = [y_max - value[1] for key, value in truck_coordinates.items() if key in placed_trucks]
        placed_trucks = [key for key, value in truck_coordinates.items() if key in placed_trucks]

    map = Image.open('map.png')
    fig = go.Figure()

    # Add trace for truck spots
    fig.add_trace(
        go.Scatter(x=truck_spot_x, y=truck_spot_y,
                   hovertemplate=trucks,
                   name="Open truck spots",
                   mode='markers',
                   marker_color='rgba(135, 206, 250, 0.0)',
                   marker_line_color='darkblue',
                   marker_line_width=2,
                   marker_size=10
                   )
    )

    # Add trace for placed trucks
    if placed_trucks:
        fig.add_trace(
            go.Scatter(x=placed_truck_spot_x, y=placed_truck_spot_y,
                       hovertemplate=placed_trucks,
                       name="Truck added to the map",
                       mode='markers',
                       marker_color='darkblue',
                       marker_line_color='darkblue',
                       marker_line_width=2,
                       marker_size=10
                       )
        )

    # Add trace for buildings
    fig.add_trace(
        go.Scatter(x=building_x, y=building_y,
                   hovertemplate=building_names,
                   name="Buildings with customer demand",
                   mode='markers',
                   marker_color='red',
                   marker_opacity=0.5,
                   marker_line_width=0,
                   marker_size=demand
                   )
    )

    # Add minimap image
    fig.add_layout_image(
            dict(
                source=map,
                xref="x",
                yref="y",
                x=0,
                y=550,
                sizex=500,
                sizey=550,
                sizing="stretch",
                opacity=0.9,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="simple_white")
    fig.update_xaxes(range=[0, 500], visible=False)
    fig.update_yaxes(range=[0,550], visible=False,
                    scaleanchor = "x",scaleratio = 1)
    fig.update_layout(showlegend=True)

    fig.update_layout(
        title="Burrito Optimization Game Map",
    )

    fig.show()



In [7]:
#from show_map_local import show_map

show_map(buildings, building_names, building_coordinates, demand, truck_coordinates)

# 2. Data Generation

## 2.1 Generate Preference Matrix

We regenerate the scaled_demand data(the deal between customer and truck) by make a preference matrix with a exp funtion of distance.

In [8]:
import math
def generate_scaled_demand(df_scaled,df_demand_node):
    """
    df_scaled: the demand-truck pairs dataset
    df_demand_node: the demand node dataset
    """
    df_rescaled = df_scaled.copy()

    #del(dict)
    my_dict = dict(zip(df_demand_node['index'], df_demand_node['demand']))
    df_rescaled['demand'] = df_rescaled['demand_node_index'].map(my_dict)

    # Step1:Calculate the total distance of each demand node for all truck node
    distance_sums = df_rescaled.groupby('demand_node_index')['distance'].transform('sum')

    # Step2:Calculate the value of new column
    beta = 10
    df_rescaled['scaled_demand'] = (np.exp(-beta * (df_rescaled['distance'] / distance_sums))) * df_rescaled['demand']
    df_rescaled['scaled_demand'] = df_rescaled['scaled_demand'].astype(int)

    df_rescaled = df_rescaled.drop(columns=['demand'])

    return df_rescaled

In [9]:
df_scaled = generate_scaled_demand(df_truck_demand_pair,df_demand)
df_scaled

,demand_node_index,truck_node_index,distance,scaled_demand
0,demand2,truck1,231.108640,20
1,demand2,truck6,201.157571,21
2,demand2,truck7,161.831991,22
3,demand2,truck8,113.778542,24
4,demand2,truck17,289.327111,18
...,...,...,...,...
220,demand51,truck36,298.355857,26
221,demand51,truck37,379.099609,22
222,demand51,truck43,408.808936,21
223,demand51,truck49,482.584417,18


# 3. LP Model and Solution

## 3.1 Load the Data

In [10]:
def parameter_read(df_problem,df_truck_node,df_demand,df_scaled):
    burrito_price = float(df_problem['burrito_price'][0])
    ingredient_cost = float(df_problem['ingredient_cost'][0])

    truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in df_truck_node.iterrows()}
    truck_spots = truck_coordinates.keys()

    buildings, building_names, building_coordinates, demand = gp.multidict({
        row['index']: [row['name'], (float(row['x']), float(row['y'])), float(row['demand'])] for ind,row in df_demand.iterrows()
    })

    building_truck_spot_pairs, distance, scaled_demand = gp.multidict({
        (row['demand_node_index'], row['truck_node_index']): [float(row['distance']), float(row['scaled_demand'])] for ind,row in df_scaled.iterrows() if float(row['scaled_demand'])>0# (building, truck_spot): distance, scaled_demand
    })

    return burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand

In [11]:
# Read data in files
burrito_price, ingredient_cost,truck_coordinates,truck_spots,buildings,building_names,building_coordinates,demand,scaled_demand = parameter_read(df_problem,df_truck_node,df_demand,df_truck_demand_pair)

## 3.2 Build Up the LP Model

In [12]:
# ================== Expand Truck Type Data ==================
# Define truck type attributes (0: small, 1: medium, 2: large)
truck_types = {
    0: {'capacity': 100, 'fixed_cost': 200},
    1: {'capacity': 150, 'fixed_cost': 250}, 
    2: {'capacity': 200, 'fixed_cost': 300}
}

def extended_LP_solution(burrito_price, ingredient_cost, truck_types,
                       truck_coordinates, truck_spots, buildings,
                       building_names, building_coordinates, demand,
                       building_truck_spot_pairs, distance, scaled_demand,
                       work_name="Extended_Solution", max_distance=500,
                       write_lp_file=True, show_model=True, is_show_map=True):
    """Complete optimization model supporting multiple truck types and visualization control"""
    # --- Initialize model ---
    model = gp.Model("Extended_Truck_Deployment")
    model.Params.OutputFlag = int(show_model)
    
    # --- Decision variables ---
    x = model.addVars(truck_types.keys(), truck_spots, vtype=GRB.BINARY, name="x")
    y = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y")
    
    # --- Objective function ---
    revenue = gp.quicksum(
        (burrito_price - ingredient_cost) * scaled_demand[i,j] * y[i,j]
        for i,j in building_truck_spot_pairs
    )
    cost = gp.quicksum(
        truck_types[k]['fixed_cost'] * x[k,j]
        for k in truck_types for j in truck_spots
    )
    model.setObjective(revenue - cost, GRB.MAXIMIZE)
   # --- Fully specified constraint formulation ---

    # --- Constraints ---
    # 1. Each customer served by at most one location
    model.addConstrs(
        (y.sum(i,'*') <= 1 for i in buildings),
        name="Single_Service"
    )
    
    # 2. Service activation constraint: Only deployed truck spots can serve customers
    model.addConstrs(
        (y[i,j] <= x.sum('*',j) for i,j in building_truck_spot_pairs),
        name="Service_Activation"
    )
    
    # 3. Distance constraint: Service distance cannot exceed max_distance
    model.addConstrs(
        (y[i,j] * distance[i,j] <= max_distance for i,j in building_truck_spot_pairs),
        name="Max_Distance"
    )
    
    # 4. Capacity constraint: Total demand at each location cannot exceed truck capacity
    model.addConstrs(
        (gp.quicksum(
            scaled_demand[i,j] * y[i,j] 
            for i in buildings if (i,j) in building_truck_spot_pairs
        ) <= gp.quicksum(
            truck_types[k]['capacity'] * x[k,j] 
            for k in truck_types
        ) for j in truck_spots),
        name="Capacity"
    )
    
    # 5. Each location can deploy at most one truck type
    model.addConstrs(
        (x.sum('*',j) <= 1 for j in truck_spots),
        name="Unique_Truck_Type"
    )
    
    # --- Model output ---
    if write_lp_file:
        model.write(f"{work_name}.lp")
    
    # --- Solve ---
    model.optimize()
    
    # --- Results display ---
    if show_model and model.status == GRB.OPTIMAL:
        print("\n=== Optimal Solution ===")
        print(f"Total profit: ₲{model.objVal:.2f}")
        
        # Truck deployment details
        deployed_trucks = [(k,j) for k,j in x.keys() if x[k,j].X > 0.5]
        for k,j in deployed_trucks:
            truck_type = ['Small', 'Medium', 'Large'][k]
            print(f" - Deployed {truck_type} truck at location {j} (Capacity: {truck_types[k]['capacity']}, Cost: ₲{truck_types[k]['fixed_cost']})")
        
        # Financial details
        print("\nFinancial breakdown:")
        print(f"Total revenue: ₲{revenue.getValue():.2f}")
        print(f"Total cost: ₲{cost.getValue():.2f}")
    
    # --- Visualization ---
    if is_show_map and model.status == GRB.OPTIMAL:
        placed_trucks = [j for k,j in x.keys() if x[k,j].X > 0.5]  # Get all deployment locations
        show_map(buildings, building_names, building_coordinates, demand, 
                truck_coordinates, placed_trucks=placed_trucks)
    
    return model.objVal

## 3.3 Running the Model

In [13]:

# ================== Application Example ==================
print("\nSolving the extanded model...")
optimal_value = extended_LP_solution(
    burrito_price, ingredient_cost, truck_types,
    truck_coordinates, truck_spots, buildings,
    building_names, building_coordinates, demand,
    building_truck_spot_pairs, distance, scaled_demand,
    work_name="Extended_Solution",
    max_distance=500,
    write_lp_file=True,
    show_model=True,
    is_show_map=True
)


Solving the extanded model...
Set parameter LicenseID to value 2652059
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 245 rows, 145 columns and 790 nonzeros
Model fingerprint: 0x6a0fb15f
Variable types: 0 continuous, 145 integer (145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 126 rows and 30 columns
Presolve time: 0.00s
Presolved: 119 rows, 115 columns, 360 nonzeros
Variable types: 0 continuous, 115 integer (115 binary)

Root relaxation: objective 7.528571e+02, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node